<a href="https://colab.research.google.com/github/AdamJelley/AdventOfCode2021/blob/main/Day8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import csv
import numpy as np

In [3]:
input_link = 'https://adventofcode.com/2021/day/8/input'
user_cookie = {'session':''} # Retrieve session cookie corresponding to user login (by inspecting cookies on data page)

In [4]:
raw_input = requests.get(input_link, cookies=user_cookie).text[:-1]
processed_input = [line.split(' | ') for line in raw_input.split('\n')]
segments_by_digit = {0:6,1:2,2:5,3:5,4:4,5:5,6:6,7:3,8:7,9:6} # From digit diagram

## Part 1

In [5]:
# Invert segments by digit
digits_by_segment = {v:[] for v in segments_by_digit.values()}
for k, v in segments_by_digit.items():
  digits_by_segment[v].append(k)
digits_by_segment

{2: [1], 3: [7], 4: [4], 5: [2, 3, 5], 6: [0, 6, 9], 7: [8]}

So digits 1, 4, 7, 8 have a unique number of segments with 2, 4, 3 and 7 respectively.

In [22]:
unsorted_processed_codes = [[line[i].split(' ') for i in range(2)] for line in processed_input]
processed_codes = [[[''.join(sorted(code)) for code in codes] for codes in line] for line in unsorted_processed_codes]

In [23]:
num_unique_segment_digits = 0

for line in processed_codes:
  for digit in line[1]:
    if len(digit) in set((2,4,3,7)):
      num_unique_segment_digits+=1
print(f'Num unique segment digits (1,4,7,8) in output: {num_unique_segment_digits}')

Num unique segment digits (1,4,7,8) in output: 278


## Part 2

In [83]:
def compute_code_mapping(line):
  mapping = {} 
  inverse_mapping = {}
  for code in line[0]: #First find mappings for digits with unique segments
    if len(code)==2:
      mapping[code]=1
      inverse_mapping[1]=code
    elif len(code)==4:
      mapping[code]=4
      inverse_mapping[4]=code
    elif len(code)==3:
      mapping[code]=7
      inverse_mapping[7]=code
    elif len(code)==7:
      mapping[code]=8
      inverse_mapping[8]=code
  for code in line[0]: # Then use these mappings to compute mappings for other digits
    if len(code)==6:
      if sum([letter in code for letter in inverse_mapping[1]])==1: #Using segment overlap with digit 1
        mapping[code]=6
        inverse_mapping[6]=code
      elif sum([letter in code for letter in inverse_mapping[4]])==4: #Using segment overlap with digit 4
        mapping[code]=9
        inverse_mapping[9]=code
      else:
        mapping[code]=0
        inverse_mapping[0]=code
    elif len(code)==5:
      if sum([letter in code for letter in inverse_mapping[1]])==2: #Using segment overlap with digit 1
        mapping[code]=3
        inverse_mapping[3]=code
      elif sum([letter in code for letter in inverse_mapping[4]])==2: #Using segment overlap with digit 4
        mapping[code]=2
        inverse_mapping[2]=code
      elif sum([letter in code for letter in inverse_mapping[4]])==3: #Using segment overlap with digit 4
        mapping[code]=5
        inverse_mapping[5]=code

  assert set(mapping.values())==set(range(10))

  return mapping

In [79]:
# Test mapping function
test_line = processed_codes[0]

mapping = compute_code_mapping(test_line)

for code in test_line[0]:
  try:
    print(code, mapping[code])
  except:
    (print(code, 'No key'))

acefg 3
abcef 5
abcdef 6
eg 1
abcefg 9
abdefg 0
efg 7
acdfg 2
bceg 4
abcdefg 8


In [80]:
def decode_output(line, mapping):
  output = ''
  for code in line[1]:
    output+=(str(mapping[code]))
  return int(output)

In [81]:
# Test decoder
test_output = decode_output(test_line, mapping)
print(test_output)

8324


In [82]:
# Compute result
sum_outputs = 0
for line in processed_codes:
  mapping = compute_code_mapping(line)
  output = decode_output(line, mapping)
  sum_outputs+=output

print(f'Sum of all decoded outputs: {sum_outputs}')

Sum of all decoded outputs: 986179
